In [ ]:
%matplotlib inline
import matplotlib.pyplot as pp
import numpy as np
from scipy import signal
from  scipy.io import wavfile
import librosa
import librosa.display
from IPython import display
from copy import deepcopy
from scipy.signal import fftconvolve

# the pra library
import pyroomacoustics as pra

In [ ]:
# A simple wrapper class for (1-channel) audio data
# data is a 1-D NumPy array containing the data
# rate is a number expressing the samples per second
class Audio:
    def __init__(self, data, rate):
        self.data = data
        self.rate = rate
    def play(self):
        return display.Audio(self.data, rate=self.rate)
    def plot_wave(self):
        librosa.display.waveplot(self.data, sr=self.rate)
    def plot_spectrum(self):
        n_fft = int(self.rate / 20)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(self.data, n_fft)), ref=np.max)
        librosa.display.specshow(D, y_axis='linear', sr=self.rate, hop_length=n_fft/4)
    @classmethod
    def fromfile(cls, fn):
        return cls(*librosa.load(fn, sr=None))

## Load Sound

In [ ]:
flute = Audio.fromfile("../sounds/flute.wav")
flute.play()

In [ ]:
symphony = Audio.fromfile("../sounds/symphony.wav")
symphony = Audio(symphony.data[:len(flute.data)], symphony.rate)
symphony.play()

In [ ]:
read1 = Audio.fromfile("../sounds/read.wav")
read1 = Audio(read1.data[:len(flute.data)], read1.rate)
read1.play()

In [ ]:
read2 = Audio.fromfile("../sounds/read2.wav")
read2 = Audio(read2.data[:len(flute.data)], read2.rate)
read2.play()

In [ ]:
background = Audio.fromfile("../sounds/background_noise.wav")
background = Audio(background.data[:len(flute.data)], background.rate)
background.play()

In [ ]:
silence = Audio(np.asarray([0.0] * len(flute.data)), flute.rate)
silence.play()

## Beamforming

In [ ]:
def beamformer(mic_locs, target_loc):
    Lg_t = 0.1 # filter size in seconds
    Lg = np.ceil(Lg_t*48000)
    
    fft_len = 512
    mics = pra.Beamformer(mic_locs, 48000, N=fft_len, Lg=Lg)
    source = pra.soundsource.SoundSource([7, 1, 2])

    mics.rake_delay_and_sum_weights(source)
    mics.filters_from_weights()
    return mics

In [ ]:
mic_locs = np.c_[
    [1, 4.5, 5],
    [1, 4.7, 5],
    [1, 4.9, 5],
    [1, 5.1, 5],
    [1, 5.3, 5],
    [1, 5.5, 5]
]
target_loc = [7, 1, 2]
mics = beamformer(mic_locs, target_loc)
mics.weights

In [ ]:
def beamforming(sound, noise):
    '''Room Parameters and Room Creation'''
    rt60 = 0.5
    room_dim = [10, 10, 10] # meters
    e_absorption, max_order = pra.inverse_sabine(rt60, room_dim)
    # m = pra.Material(energy_absorption="panel_fabric_covered_6pcf")
    room = pra.ShoeBox(room_dim, fs=48000, materials=pra.Material(e_absorption), max_order=0) #simulate perfect situation: no reflections

    '''Sound Source Creation'''
    room.add_source([7, 1, 2], signal=sound)
    room.add_source([8, 5, 5], signal=noise)

    '''Mic Array Creation'''
    mic_locs = np.c_[
        [1, 4.5, 5],
        [1, 4.7, 5],
        [1, 4.9, 5],
        [1, 5.1, 5],
        [1, 5.3, 5],
        [1, 5.5, 5]
    ]

#     mic_locs = np.c_[
#         [1, 5, 5],
#         [1, 5.5, 5],
#         [1, 5, 5.5],
#         [1, 5.5, 5.5]
#     ]

    # # center of array as column vector
    # mic_center = np.c_[[1, 5, 5]]
    # # microphone array radius
    # mic_radius = 0.05
    # # number of elements
    # mic_n = 8
    # # The GridSphere objects creates a number of points
    # # pseudo-uniformly spread on the unit sphere
    # grid = pra.doa.GridSphere(mic_n)
    # # The locations of the microphones can then be computed
    # mic_locs = mic_center + mic_radius * grid.cartesian

    # filter size (???)
    Lg_t = 0.1 # filter size in seconds
    Lg = np.ceil(Lg_t*room.fs)


    # place the beamforming micarray in the room (the beamforming class is a child class of the micarray class)
    fft_len = 512
    mics = pra.Beamformer(mic_locs, room.fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)


    # Compute DAS (delay and sum) weights
    mics.rake_delay_and_sum_weights(room.sources[0])

    '''Simulation'''
    room.compute_rir()
    room.simulate()
    return room, mics

### Flute vs. Background noise

In [ ]:
room1, mics1 = beamforming(flute.data, background.data)
fig, ax = room1.plot()

In [ ]:
### Original_mixed ###
mix_audio = Audio(room1.mic_array.signals[0, :], room1.fs)
mix_audio.play()

In [ ]:
### Beamformed ###
bf_signal = mics1.process()
bf_audio = Audio(bf_signal * np.linalg.norm(mix_audio.data) / np.linalg.norm(bf_signal.data), room1.fs)
bf_audio.play()

### Customized Flute vs. Background noise

In [ ]:
signals = mics1.signals
output = fftconvolve(mics.filters[0], signals[0])
for i in range(1, len(signals)):
    output += fftconvolve(mics.filters[i], signals[i])
output

In [ ]:
Audio(output, flute.rate).play()

In [ ]:
bf_signal